In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras import metrics

Using TensorFlow backend.


In [2]:
test_features = pd.read_csv("dengue_features_test.csv", index_col=[0,1,2]) 
train_features = pd.read_csv("dengue_features_train.csv", index_col=[0,1,2]) 
train_labels = pd.read_csv("dengue_labels_train.csv", index_col=[0,1,2]) 
submission_format = pd.read_csv("submission_format.csv", index_col=[0,1,2]) 

In [3]:
select_features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c', 
                 'station_min_temp_c', 'week_start_date']

## Only IQ Columns

In [4]:
# Seperate data for San Juan
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc["iq"]
iq_test_features = test_features.loc["iq"]

In [5]:
#removing NaNs
iq_train_features.fillna(0, inplace=True)
iq_test_features.fillna(0, inplace=True)

c:\users\milla\projects\databears\lib\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [6]:
iq_train_features.set_index("week_start_date")
iq_test_features.set_index("week_start_date")

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,
2010-07-02,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,299.214286,296.118571,307.9,293.7,34.11,92.581429,82.29,17.654286,9.428571,27.440,10.760,33.8,21.5,11.2
2010-07-09,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,299.464286,294.831429,307.7,292.2,9.10,83.885714,25.30,16.320000,10.157143,27.025,9.625,33.0,21.2,8.9
2010-07-16,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,298.964286,296.417143,306.4,292.5,61.09,92.057143,62.14,18.030000,9.557143,26.950,10.350,33.4,21.6,22.6
2010-07-23,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,296.642857,293.638571,306.4,289.3,19.60,88.970000,47.80,15.394286,7.828571,26.900,9.700,33.3,14.2,4.8
2010-07-30,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,300.000000,293.892857,308.1,290.5,18.93,78.610000,56.30,15.468571,11.771429,27.050,11.850,33.5,16.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-05-28,0.301471,0.380029,0.280629,0.383186,41.12,297.774286,298.964286,295.638571,305.5,292.7,67.60,89.990000,41.12,17.185714,10.100000,27.400,9.050,32.6,21.8,33.0
2013-06-04,0.247600,0.296343,0.285371,0.350357,71.52,297.167143,298.328571,295.845714,306.3,291.6,45.70,93.891429,71.52,17.448571,9.657143,27.520,10.720,33.8,21.4,68.0
2013-06-11,0.238729,0.251029,0.252586,0.249771,78.96,295.831429,296.607143,294.894286,304.6,290.7,45.22,94.967143,78.96,16.410000,7.385714,27.200,10.075,32.6,21.6,93.2


In [7]:
#removing NaNs - test

iq_train_features.isnull().sum()
# iq_train_features

week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
dtype: int64

In [8]:
iq_train_labels["week_start_date"] = iq_train_features["week_start_date"]

In [9]:
iq_train_labels.set_index("week_start_date")

,total_cases
week_start_date,
2000-07-01,0
2000-07-08,0
2000-07-15,0
2000-07-22,0
2000-07-29,0
...,...
2010-05-28,5
2010-06-04,8
2010-06-11,1


In [10]:
iq_train_features = iq_train_features[select_features]

In [11]:
iq_train_features

reanalysis_specific_humidity_g_per_kg  \
year weekofyear                                          
2000 26                                      16.651429   
     27                                      16.862857   
     28                                      17.120000   
     29                                      14.431429   
     30                                      15.444286   
...                                                ...   
2010 21                                      18.485714   
     22                                      18.070000   
     23                                      17.008571   
     24                                      16.815714   
     25                                      17.355714   

                 reanalysis_dew_point_temp_k  station_avg_temp_c  \
year weekofyear                                                    
2000 26                           295.184286           26.400000   
     27                           295.358571           26.900000   
     28                           295.622857           26.800000   
     29                           292.797143           25.766667   
     30                           293.957143           26.600000   
...                                      ...                 ...   
2010 21                           296.825714           28.633333   
     22                           296.452857           27.433333   
     23                           295.501429           24.400000   
     24                           295.324286           25.433333   
     25                           295.807143           27.475000   

                 station_min_temp_c week_start_date  
year weekofyear                                      
2000 26                        20.7      2000-07-01  
     27                        20.8      2000-07-08  
     28                        20.7      2000-07-15  
     29                        14.7      2000-07-22  
     30                        19.1      2000-07-29  
...                             ...             ...  
2010 21                        22.4      2010-05-28  
     22                        21.7      2010-06-04  
     23                        19.2      2010-06-11  
     24                        21.0      2010-06-18  
     25                        22.2      2010-06-25  

[520 rows x 5 columns]

In [12]:
# data normalization
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler(feature_range=(0,1))
sc_Y = MinMaxScaler(feature_range=(0,1))
iq_train_features_scaled = sc_X.fit_transform(iq_train_features.drop(columns = ["week_start_date"]))
iq_train_labels_scaled = sc_Y.fit_transform(np.reshape(iq_train_labels["total_cases"].tolist(), (520, 1)))

In [13]:
iq_train_features_scaled

array([[0.81379599, 0.98905775, 0.85714286, 0.8553719 ],
       [0.82412902, 0.98964172, 0.87337662, 0.85950413],
       [0.83669622, 0.99052725, 0.87012987, 0.8553719 ],
       ...,
       [0.83125044, 0.99012038, 0.79220779, 0.79338843],
       [0.82182504, 0.98952684, 0.82575758, 0.8677686 ],
       [0.84821616, 0.99114472, 0.89204545, 0.91735537]])

In [14]:
iq_train_labels_scaled.shape

(520, 1)

In [15]:
iq_train_labels

total_cases week_start_date
year weekofyear                             
2000 26                    0      2000-07-01
     27                    0      2000-07-08
     28                    0      2000-07-15
     29                    0      2000-07-22
     30                    0      2000-07-29
...                      ...             ...
2010 21                    5      2010-05-28
     22                    8      2010-06-04
     23                    1      2010-06-11
     24                    1      2010-06-18
     25                    4      2010-06-25

[520 rows x 2 columns]

In [16]:
#incoperating time steps into data

X_train = []
y_train = []
for i in range(0, len(iq_train_features_scaled)):
    X_train.append(iq_train_features_scaled[i,:])
    y_train.append(iq_train_labels_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [17]:
X_train.shape

(520, 1, 4)

In [18]:
X_train[1:].shape

(519, 1, 4)

In [61]:
#Model
split = 400

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Bidirectional, Activation

model = Sequential()
model.add(Bidirectional(LSTM(units=50,return_sequences=True,input_shape=(1, X_train.shape[2]))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50)))
model.add(Dropout(0.2))
model.add(Dense(1))
# model.add(Activation('softmax'))
model.compile(optimizer='adam',loss='mean_squared_error' )
model.fit(X_train[:split],y_train[:split],epochs=100,batch_size=32, 
          validation_data=(X_train[split:], y_train[split:]))
model.save('LSTM_IQ_selectfeatures_tinkered.h5')

Train on 400 samples, validate on 120 samples
Epoch 1/100
400/400 [==============================] - 3s 7ms/step - loss: 0.8871 - val_loss: 0.8667
Epoch 2/100
400/400 [==============================] - 0s 222us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 3/100
400/400 [==============================] - 0s 197us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 4/100
400/400 [==============================] - 0s 197us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 5/100
400/400 [==============================] - 0s 255us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 6/100
400/400 [==============================] - 0s 237us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 7/100
400/400 [==============================] - 0s 227us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 8/100
400/400 [==============================] - 0s 203us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 9/100
400/400 [==============================] - 0s 201us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 10/100
400/400 [=======

400/400 [==============================] - 0s 199us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 80/100
400/400 [==============================] - 0s 203us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 81/100
400/400 [==============================] - 0s 227us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 82/100
400/400 [==============================] - 0s 200us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 83/100
400/400 [==============================] - 0s 220us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 84/100
400/400 [==============================] - 0s 230us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 85/100
400/400 [==============================] - 0s 208us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 86/100
400/400 [==============================] - 0s 208us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 87/100
400/400 [==============================] - 0s 219us/step - loss: 0.8871 - val_loss: 0.8667
Epoch 88/100
400/400 [==============================] - 0s 218us/step - loss:

In [62]:
# if model exist, just load it
# model = load_model('LSTM_IQ.h5')

In [63]:
iq_test_features = iq_test_features[select_features]

In [64]:
#pre processing for test data
iq_test_features_scaled = sc_X.fit_transform(iq_test_features.drop(columns = ["week_start_date"]))

In [65]:
iq_test_features_scaled.shape

(156, 4)

In [66]:
X_test = []
for i in range(0, len(iq_test_features_scaled)):
    X_test.append(iq_test_features_scaled[i,:])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [67]:
predictions = model.predict(X_test)

In [68]:
predictions

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [69]:
predictions = sc_Y.inverse_transform(predictions)

In [70]:
iq_submission = submission_format.loc["iq"]
# sj_submission["total_cases"] = predictions
# sj_submission = predictions_submission.reset_index()
# sj_submission.insert(0, 'city', 'sj')

# predictions.shape
iq_submission

total_cases
year weekofyear             
2010 26                    0
     27                    0
     28                    0
     29                    0
     30                    0
...                      ...
2013 22                    0
     23                    0
     24                    0
     25                    0
     26                    0

[156 rows x 1 columns]

In [71]:
iq_submission["total_cases"] = predictions

In [72]:
iq_submission.insert(0, 'city', 'iq')

In [73]:
iq_submission.reset_index()

,year,weekofyear,city,total_cases
0,2010,26,iq,116.0
1,2010,27,iq,116.0
2,2010,28,iq,116.0
3,2010,29,iq,116.0
4,2010,30,iq,116.0
...,...,...,...,...
151,2013,22,iq,116.0
152,2013,23,iq,116.0
153,2013,24,iq,116.0
154,2013,25,iq,116.0


In [74]:
iq_submission.to_csv("LSTMiq_selectfeatures_tinkered.csv")